In [30]:
import pandas as pd
import numpy as np

In [2]:
a = []

In [4]:
a.append('No "Payment details 2" file need to be added')

In [10]:
pd.DataFrame(a, columns=['log'])

,log
0,"No ""Payment details"" file need to be added"
1,"No ""Payment details 2"" file need to be added"


In [18]:
a.append(pd.Timestamp.now())

In [32]:
pd.Timestamp.now()

Timestamp('2022-10-27 16:02:11.672543')

In [23]:
b= [['payment', pd.Timestamp.now()]]

In [25]:
b.append(['dim_agent', pd.Timestamp.now()])

In [33]:
test = pd.DataFrame(b, columns=['table', 'time_updated'])

In [37]:
test['time_updated']

0   2022-10-27 15:58:48.452994
1   2022-10-27 15:59:06.816978
2   2022-10-27 15:59:13.417844
Name: time_updated, dtype: datetime64[ns]

In [44]:
# import necessary materials
import sqlite3
import pandas as pd
import glob
import os
from tqdm import tqdm
import datetime
from datetime import timedelta
pd.options.mode.chained_assignment = None

def file_name_modified(path:str):
    all_dir_files = glob.glob(path + '/*')
    m_time = []

    for dir_file in all_dir_files:

        # file modification timestamp of a file
        f_m_time = os.path.getmtime(dir_file)
        # convert timestamp into DateTime object
        f_dt_m = datetime.datetime.fromtimestamp(f_m_time)
        m_time += [f_dt_m]

    files_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -1]
    table_name = pd.Series(all_dir_files).str.split('\\', expand=True).iloc[:, -2]
    return pd.DataFrame({'file_name':files_name, 'modified_time':m_time,
                         'table_name':table_name, 'dir_file':all_dir_files})

# create connection to sqlite
conn = sqlite3.connect(r"D:\NMT\OneDrive\Viettravel Airline\Database\VTA_RM.db")
c = conn.cursor()
warining_msg = '*****WARNING***** cannot add new rows to SQLite'


cargo = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\cargo'
flown_aircraft_leg = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\flown_aircraft_leg'
inflow_cash = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\inflow_cash'
pax_revenue = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\pax_revenue'
payment_detail = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\payment_detail'
reservation = r'D:\NMT\OneDrive\Viettravel Airline\Database\fact\reservation'

dim_agent = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_agent'
dim_calendar = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_calendar'
dim_fare_code = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_fare_code'
dim_routes = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_routes'
dim_slot_time = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\dim_slot_time'
flight_type = r'D:\NMT\OneDrive\Viettravel Airline\Database\dim\flight_type'


dir_list = [cargo, flown_aircraft_leg, inflow_cash, pax_revenue, payment_detail, reservation,
        dim_agent, dim_calendar, dim_routes, dim_fare_code, dim_slot_time, flight_type]
frame_list = []

for dir_x in dir_list:
    frame_list.append(file_name_modified(dir_x))

new_log_table = pd.concat(frame_list, axis=0, ignore_index=True)

query = """
SELECT *
    FROM log_table
"""

previous_log_table = pd.read_sql_query(query, conn)
previous_log_table['modified_time'] = pd.to_datetime(previous_log_table['modified_time'])

diff_table = pd.concat([previous_log_table, new_log_table]).drop_duplicates(ignore_index=True, keep=False)
keep_table = pd.concat([previous_log_table, diff_table, diff_table]).drop_duplicates(ignore_index=True, keep=False)
remove_table = pd.concat([previous_log_table, keep_table]).drop_duplicates(ignore_index=True, keep=False)
add_table = pd.concat([new_log_table, keep_table]).drop_duplicates(ignore_index=True, keep=False)

In [49]:
table = remove_table[remove_table['table_name'] == 'inflow_cash']

In [53]:
table['table_name'].values[0]

'inflow_cash'